<a href="https://colab.research.google.com/github/grissharrisdennis/Machine-learning-Projects/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass

In [ ]:
!pip install tensorflow-datasets

In [ ]:

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
columns=['label','message']

In [ ]:
train_data=pd.read_csv(train_file_path,names=columns,sep='\t',header=None)
test_data=pd.read_csv(test_file_path,names=columns,sep='\t',header=None)

In [ ]:
encoder = tf.keras.layers.TextVectorization(max_tokens=1000, output_mode='int', output_sequence_length=100)
encoder.adapt(train_data['message'].values)

# Encode the labels ('ham' and 'spam')
label_encoder = {'ham': 0, 'spam': 1}
train_data['label'] = train_data['label'].map(label_encoder)
test_data['label'] = test_data['label'].map(label_encoder)

# Prepare the input and output data
X_train = encoder(train_data['message'].values)
y_train = train_data['label'].values
X_test = encoder(test_data['message'].values)
y_test = test_data['label'].values

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=1000, output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
def predict_message(message):
    message = np.array([message])
    encoded_mes = encoder(message)
    prediction = model.predict(encoded_mes)
    label = "ham" if prediction < 0.5 else "spam"
    predict_list=[float(prediction),label]
    return predict_list

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
